In [1]:
import torch
import torchvision
import torch.nn as nn 
from torch.nn import functional as F 
from VGG16_teacher import VGG           # Import the teacher model
                                        # Teacher model: VGG 16 

In [2]:
random_seed = 1
torch.manual_seed(random_seed)

# Student Network
## Student Network: 5 layer convolutional network (4 conv + 1 fc) 

In [ ]:
class five_layer_model(nn.Module):
    def __init__(self):
        super(five_layer_model, self).__init__()

    def forward(self, x):
        return 

# Teacher Network: Load weight from saved file

In [ ]:
student_model = five_layer_model()
student_model = student_model.cuda()

teacher_model = VGG('VGG16')
teacher_model = teacher_model.cuda()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
teacher_model.load_state_dict(torch.load('VGG16_teacher.pth'))

# New Loss function 
## KL divergence + Cross entropy loss

In [ ]:
# Use "nn.KLDivLoss()" and "nn.CrossEntropyLoss" the make new loss 
# Use softmax and log softmax: F.softmax(), F.log_softmax()

def distillation(student_scores, true_labels, teacher_scores, T, alpha):    

In [ ]:
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
import time

optimizer = optim.SGD(student_model.parameters(), lr = """???""", momentum = 0.9)
criterion = nn.CrossEntropyLoss()
step_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[""???""], gamma = """???""")

# Train and Test code
## There is blank in Training code. 

## - Teacher model: Evaluation mode
### Weights should not be updated
## - Student model: Training mode.  
### Have to update weights 

In [ ]:
def train(epoch):
    # Student model Train mode 
    # Teacher model Test mode 
    running_loss = 0
    for batch_idx, data in enumerate(train_loader): 
        image, label = data
        image = image.cuda()
        labels = label.cuda()
        optimizer.zero_grad()

        student_output = # blank 
        teacher_output = # blank 
        teacher_output = teacher_output.detach()

        loss = # blank
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if batch_idx % 30 ==29:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * 128, len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(student_model.state_dict(), './distillation_model.pth')

In [ ]:
def test(model, append = True):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            test_loss += loss.item()
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss/(batch_idx+1), correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
    if append:     
        test_losses.append(test_loss/(batch_idx+1))
        accuracy.append(100. * correct / len(test_loader.dataset))

# Load the CIFAR 10 data which downloaded yesterday. 
## Change the "root "

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.CIFAR10(root = 'YOUR_PATH', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                                 torchvision.transforms.RandomCrop(32, padding = 4),
                                 torchvision.transforms.RandomHorizontalFlip(),
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                             ])),
  batch_size=128, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.CIFAR10(root = 'YOUR_PATH', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                             ])),
  batch_size=128, shuffle=True)

In [ ]:
epochs = 50
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(epochs + 1)]
accuracy = []

# Check teacher model's accuracy 

In [ ]:
test(teacher_model, append=False)

# Train Step
# Teacher model's Accuracy should be same every time 

In [ ]:
test(student_model, append = True)
since = time.time()
for epoch in range(1, epochs + 1):
    train(epoch)
    test(student_model, append = True)
    test(teacher_model, append  = False)
    step_lr_scheduler.step()
finish = time.time()
print('Training Finished in {}min'.format((finish - since)//60))

# Plot Training Loss and Testing Loss 

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
plt.plot()

# Plot Test Accuracy 

In [ ]:
plt.figure()
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.plot(range(len(accuracy)), accuracy, color = 'blue')
plt.plot()

# Summary of the models 
## if no module named Error, go to cmd, pip install torchsummary 

In [ ]:
from torchsummary import summary 

print('The number of parameters in Student model')
summary(student_model, (3, 32, 32))

print('The number of parameters in Teacher model')
summary(teacher_model, (3, 32, 32))